In [10]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import random

import time
import csv

import os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [12]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.discogs.com/sell/list?sort=price%2Cdesc&limit=100&year1=1900&year2=1970&format=Vinyl&price=over40&genre=Jazz&currency=USD&style=Hard+Bop&page=3#more%3Dyear")

# add country label format

with open('recorddata.csv', 'w',newline='') as csvfile:
    file = csv.writer(csvfile)
    # make headers
    file.writerow(['Artist_Album', 'Genre', 'Release_Date', 'Style', 'Rate_Haves_Wants', 'Media_Condition',
                     'Sleeve_Condition', 'Seller_Rating', 'Price'])
    count = 3
    
    while count < 7:
        
        # find all links on album marketplace page and store in list

        result_elements = '//a[contains(@href, "/sell/item/")]'

        albums = []

        albumdriver = driver.find_elements_by_xpath(result_elements)

        for url in albumdriver:
            albums.extend([url.get_attribute('href')])
        
        # get rid of duplicates

        albumsclean = [album.split('?', 1)[0] for album in albums]
        albumurls = set(albumsclean)

        # scrape info from album page

        for album in albumurls:

            driver.get(album)

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                artist_album = driver.find_element_by_xpath("//h1[contains(@id, 'profile_title')]").text.strip()
            except:
                artist_album = np.nan
        
            try:
                genre = driver.find_element_by_xpath("//div[contains(text(), 'Genre:')]/following-sibling::div").text.strip()
            except:
                genre = np.nan

            try:
                release_date = driver.find_element_by_xpath("//div[contains(text(), 'Released:')]/following-sibling::div").text.strip()
            except:
                release_date = np.nan

            try:
                style = driver.find_element_by_xpath("//div[contains(text(), 'Style:')]/following-sibling::div").text.strip()
            except:
                style = np.nan

            try:
                rate_haves_wants = driver.find_element_by_xpath("//a[contains(@class, 'button-blue')]/following-sibling::div").text.strip()
            except:
                rate_haves_wants = np.nan

            try:
                m_condition = driver.find_element_by_xpath("//strong[contains(text(), 'Media:')]/following-sibling::span").text.strip()
            except:
                m_condition = np.nan
    
            try:
                sleeve = driver.find_element_by_xpath("//strong[contains(text(), 'Sleeve:')]")
                s_condition = sleeve.find_element_by_xpath('..').text.strip()
            except:
                s_condition = np.nan

            try:
                seller_rating = driver.find_element_by_xpath("//span[@class='star_rating']/following-sibling::strong").text.strip()
            except:
                seller_rating = np.nan

            try:
                price = soup.find(class_='price').text.strip()
            except:
                price = np.nan

            observation = [artist_album, genre, release_date, style, rate_haves_wants, m_condition, 
                           s_condition, seller_rating, price]

            file.writerow(observation)

            time.sleep(.5+2*random.random())
            
        # go to next page in the marketplace
        
        #nextpage = "https://www.discogs.com/sell/list?sort=listed%2Casc&currency=USD&limit=25&page=" + str(count+1) + "&format=Vinyl"
        nextpage = "https://www.discogs.com/sell/list?sort=price%2Cdesc&limit=100&year1=1900&year2=1970&format=Vinyl&price=over40&genre=Jazz&currency=USD&style=Hard+Bop&page=" + str(count+1) + "#more%3Dyear"
        driver.get(nextpage)
        
        count = count + 1

KeyboardInterrupt: 